# Climate, Environment & Pollution
## Preprocessing & Merge 

In [1]:
import os
import pandas as pd
pd.set_option('display.max_rows', 150)
#pd.options.mode.chained_assignment = None
from eurostat import *
import numpy as np

## Cooling and heating degree days by NUTS 2 regions - annual data
https://ec.europa.eu/eurostat/cache/metadata/en/nrg_chdd_esms.htm

In [3]:
heatdays = get_data_df('nrg_chddr2_a', flags=False)
heatdays = heatdays.iloc[:,:11].rename(columns={'geo\\time': ' NUTS 2'})

# Merge on all entries which are also in the target variable cities to extract only the interesting cities
target_cities = pd.read_csv("Cities_with_codes.csv")
heatdays_nuts = pd.merge(target_cities, heatdays, on=[' NUTS 2'])


print('any missing values: ' + str(heatdays_nuts.isna().any().any()))
heatdays_nuts.head(2)

any missing values: False


,City,City Code,NUTS 2,Country,unit,indic_nrg,2019,2018,2017,2016,2015,2014,2013,2012
0,Amsterdam,NL002C1,NL32,NL,NR,CDD,33.41,18.95,2.40,9.78,12.22,6.81,7.28,6.85
1,Amsterdam,NL002C1,NL32,NL,NR,HDD,2476.15,2524.38,2501.11,2642.52,2601.07,2257.99,3005.27,2818.95


In [4]:
heatdays_cdd = heatdays_nuts.iloc[np.where(heatdays_nuts.indic_nrg == 'CDD')]
heatdays_cdd.to_csv('../../data/heatdays_cdd.csv')
heatdays_hdd = heatdays_nuts.iloc[np.where(heatdays_nuts.indic_nrg == 'HDD')]
heatdays_cdd.to_csv('../../data/heatdays_hdd.csv')

In [6]:
print(heatdays_hdd.shape)
heatdays_hdd.head(2)

(61, 14)


,City,City Code,NUTS 2,Country,unit,indic_nrg,2019,2018,2017,2016,2015,2014,2013,2012
1,Amsterdam,NL002C1,NL32,NL,NR,HDD,2476.15,2524.38,2501.11,2642.52,2601.07,2257.99,3005.27,2818.95
3,Antwerp,BE002C1,BE2,BE,NR,HDD,2372.84,2392.04,2396.57,2520.37,2451.94,2117.72,2846.68,2596.03


## Number and capacity of recovery and diswastecap_engrecposal facilities by NUTS 2 regions
Energy recovery (RCV_E): Operation R1;
The dataset on waste treatment facilities is broken down into the measurement variable (number of facilities, capacity), the waste operation and NUTS2 regions.

https://ec.europa.eu/eurostat/cache/metadata/en/env_wasgt_esms.htm

In [22]:
wastecap = get_data_df('env_wasfac', flags=False)
wastecap = wastecap.rename(columns={'geo\\time': ' NUTS 2'})
wastecap_nuts = pd.merge(target_cities, wastecap, on=[' NUTS 2'])

## filter for Energy recovery (RCV_E) measure and CAP (capacity)
wastecap_nuts = wastecap_nuts.iloc[np.where(wastecap_nuts.wst_oper == 'RCV_E')]
wastecap_engrec = wastecap_nuts.iloc[np.where(wastecap_nuts.indic_env == 'CAP')]

print(wastecap_engrec.shape)
wastecap_engrec.head(3)

## TODO interpolate odd years

(76, 14)


,City,City Code,NUTS 2,Country,indic_env,wst_oper,2018,2016,2014,2012,2010,2008,2006,2004
3,Amsterdam,NL002C1,NL32,NL,CAP,RCV_E,2333000.0,2243000.0,2243000.0,2243000.0,975000.0,0.0,93.0,3840.0
18,Ankara,TR001C1,TR51,TR,CAP,RCV_E,NaN,NaN,NaN,NaN,NaN,NaN,0.0,NaN
34,Antwerp,BE002C1,BE2,BE,CAP,RCV_E,3256785.0,3195547.0,3300195.0,3641982.0,1763270.0,1763270.0,1849801.0,5391947.0


In [23]:
# Check for missing values
print('Missing values for columns:')
def NaN_percent(df, column_name):
    row_count = df[column_name].shape[0]
    empty_values = row_count - df[column_name].count()
    return (100.0*empty_values)/row_count
for i in list(wastecap_engrec):
    print("%s: %.2f%%" % (i, NaN_percent(wastecap_engrec,i)))


Missing values for columns:
City: 0.00%
 City Code: 0.00%
 NUTS 2: 0.00%
 Country: 0.00%
indic_env: 0.00%
wst_oper: 0.00%
2018: 10.53%
2016: 11.84%
2014: 17.11%
2012: 22.37%
2010: 28.95%
2008: 40.79%
2006: 32.89%
2004: 68.42%


In [25]:
## TODO interpolate entire columns for odd years

#    make empty columns, then rearrange order, filter < 2012,  flip to ascending years
#    then interpolate

wastecap_engrec.interpolate(method='linear').head(8)

,City,City Code,NUTS 2,Country,indic_env,wst_oper,2018,2016,2014,2012,2010,2008,2006,2004
3,Amsterdam,NL002C1,NL32,NL,CAP,RCV_E,2333000.0,2243000.0,2243000.0,2243000.0,975000.0,0.000000e+00,93.0,3.840000e+03
18,Ankara,TR001C1,TR51,TR,CAP,RCV_E,2794892.5,2719273.5,2771597.5,2942491.0,1369135.0,8.816350e+05,0.0,2.697894e+06
34,Antwerp,BE002C1,BE2,BE,CAP,RCV_E,3256785.0,3195547.0,3300195.0,3641982.0,1763270.0,1.763270e+06,1849801.0,5.391947e+06
49,Athens,EL001C1,EL30,EL,CAP,RCV_E,34757.0,43276.0,43276.0,43277.0,43756.0,6.449300e+04,1263855.5,2.695974e+06
64,Barcelona,ES002C1,ES51,ES,CAP,RCV_E,2163268.0,2148592.0,2148592.0,1343814.0,808764.0,5.132867e+04,677910.0,0.000000e+00
79,Valencia,ES003C1,ES51,ES,CAP,RCV_E,2163268.0,2148592.0,2148592.0,1343814.0,808764.0,3.816433e+04,677910.0,0.000000e+00
94,Belfast,UK012C1,UKN0,UK,CAP,RCV_E,376000.0,196000.0,36000.0,32375.0,18140.0,2.500000e+04,0.0,5.428571e+03
108,Bergen,NO002C1,NO05,NO,CAP,RCV_E,350000.0,350000.0,350000.0,355500.0,387500.0,3.466000e+05,228600.0,1.085714e+04


# More potential data
internet https://ec.europa.eu/eurostat/cache/metadata/en/isoc_i_esms.htm

soil cover - ef_mp_soil https://ec.europa.eu/eurostat/cache/metadata/en/ef_esms.htm #massive 2.5M rows

waste - env_rwas_gen https://ec.europa.eu/eurostat/cache/metadata/en/env_rwas_gen_esms.htm

air transport - tran_r_avpa_nm https://ec.europa.eu/eurostat/cache/metadata/en/reg_tran_esms.htm

(tgs00026 disposable income of private households)

econ - nama_10r_2emhrw https://ec.europa.eu/eurostat/cache/metadata/en/reg_eco10_esms.htm



In [32]:
waste = get_data_df('env_rwas_gen', flags=False)
waste.head()

,wst_oper,unit,geo\time,2013,2012,2011,2010,2009,2008,2007,2006,2005,2004,2003,2002,2001,2000
0,DSP_I,THS_T,AT,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,NaN,NaN,NaN,NaN
1,DSP_I,THS_T,AT11,0.0,0.0,0.0,0.0,0.0,0.0,0.0,NaN,NaN,0.0,NaN,NaN,NaN,NaN
2,DSP_I,THS_T,AT12,0.0,0.0,0.0,0.0,0.0,0.0,0.0,NaN,NaN,0.0,NaN,NaN,NaN,NaN
3,DSP_I,THS_T,AT13,0.0,0.0,0.0,0.0,0.0,0.0,0.0,NaN,NaN,0.0,NaN,NaN,NaN,NaN
4,DSP_I,THS_T,AT21,0.0,0.0,0.0,0.0,0.0,0.0,0.0,NaN,NaN,0.0,NaN,NaN,NaN,NaN


In [35]:
econ = get_data_df('nama_10r_2emhrw', flags=False)
econ.head(5)

,unit,wstatus,nace_r2,geo\time,2018,2017,2016,2015,2014,2013,...,2004,2003,2002,2001,2000,1999,1998,1997,1996,1995
0,THS,EMP,A,AT,NaN,406750.0,412850.0,415440.0,435520.0,434110.0,...,602090.0,618190.0,621500.0,632300.0,650890.0,NaN,NaN,NaN,NaN,NaN
1,THS,EMP,A,AT1,NaN,119800.0,120980.0,118450.0,124980.0,125140.0,...,176740.0,182850.0,181680.0,185900.0,196790.0,NaN,NaN,NaN,NaN,NaN
2,THS,EMP,A,AT11,NaN,18940.0,18990.0,19180.0,20330.0,20010.0,...,30760.0,32130.0,33130.0,34850.0,36650.0,NaN,NaN,NaN,NaN,NaN
3,THS,EMP,A,AT12,NaN,98760.0,100020.0,97280.0,102530.0,103090.0,...,143680.0,148400.0,146190.0,148560.0,157500.0,NaN,NaN,NaN,NaN,NaN
4,THS,EMP,A,AT13,NaN,2100.0,1970.0,1990.0,2120.0,2040.0,...,2300.0,2320.0,2360.0,2490.0,2640.0,NaN,NaN,NaN,NaN,NaN


In [36]:
transp = get_data_df('tran_r_avpa_nm', flags=False)
transp.head(5)

,tra_meas,unit,geo\time,2019,2018,2017,2016,2015,2014,2013,...,2002,2001,2000,1999,1998,1997,1996,1995,1994,1993
0,PAS_CRD,THS_PAS,AT,35644.0,31138.0,28326.0,27181.0,26754.0,26378.0,25749.0,...,14944.0,7640.0,7677.0,NaN,6857.0,6323.0,5885.0,5528.0,4969.0,4644.0
1,PAS_CRD,THS_PAS,AT1,31635.0,27025.0,24333.0,23318.0,22740.0,22473.0,22041.0,...,11911.0,5886.0,5916.0,NaN,5261.0,4800.0,4481.0,4190.0,3760.0,3496.0
2,PAS_CRD,THS_PAS,AT12,31635.0,27025.0,24333.0,23318.0,22740.0,22473.0,22041.0,...,11911.0,5886.0,5916.0,NaN,5261.0,4800.0,4481.0,4190.0,3760.0,3496.0
3,PAS_CRD,THS_PAS,AT2,1245.0,1256.0,1169.0,1179.0,1186.0,1115.0,1131.0,...,1030.0,474.0,478.0,NaN,422.0,407.0,361.0,350.0,325.0,298.0
4,PAS_CRD,THS_PAS,AT21,210.0,230.0,216.0,195.0,230.0,226.0,256.0,...,258.0,112.0,115.0,NaN,112.0,110.0,99.0,100.0,102.0,100.0


In [38]:
## disposable income
dincome = get_data_df('tgs00026', flags=False)
dincome.head(5)

,unit,direct,na_item,geo\time,2007,2008,2009,2010,2011,2012,2013,2014,2015,2016,2017,2018
0,PPS_HAB,BAL,B6N,AT11,19900.0,19700.0,19200.0,19600.0,19700.0,20900.0,20500.0,21200.0,22100.0,22000.0,22300.0,NaN
1,PPS_HAB,BAL,B6N,AT12,20800.0,20700.0,20200.0,20600.0,20900.0,21800.0,21600.0,22000.0,22600.0,22700.0,22600.0,NaN
2,PPS_HAB,BAL,B6N,AT13,20800.0,20600.0,20100.0,20200.0,20400.0,21000.0,20500.0,20800.0,21200.0,20900.0,20700.0,NaN
3,PPS_HAB,BAL,B6N,AT21,19300.0,19200.0,18700.0,19000.0,19200.0,20200.0,20000.0,20600.0,21200.0,21100.0,21300.0,NaN
4,PPS_HAB,BAL,B6N,AT22,19500.0,19500.0,18900.0,19300.0,19600.0,20500.0,20200.0,20800.0,21400.0,21300.0,21500.0,NaN


# Scraping Polution index from numbeo
https://www.numbeo.com/pollution/region_rankings.jsp?title=2019&region=150

adapted response bs scraper for pollution

In [50]:
!python Pollution_Scraper.py

In [51]:
!ls ../../data

 energy_simlp_nrg_100a.tsv	  Pollution_2017	 Quality_of_life_2017
'energy_simlp_nrg_100a .tsv.gz'   Pollution_2018	 Quality_of_life_2018
 heatdays_cdd.csv		  Pollution_2019	 Quality_of_life_2019
 heatdays_hdd.csv		  Pollution_2020	 Quality_of_life_2020
 Pollution_2012			  Quality_of_life_2012	 t2020_30.tsv
 Pollution_2013			  Quality_of_life_2013	 t2020_31.tsv
 Pollution_2014			  Quality_of_life_2014	 t2020_rl100+ESTAT.tsv
 Pollution_2015			  Quality_of_life_2015	 t2020_rn210.tsv
 Pollution_2016			  Quality_of_life_2016	 urb_cenv.tsv


In [49]:
#!python Response_Variable.py